# Homework 4: Language Modelling in Hangman

Student Name:Mao Yundong

Student ID:882542

Python version used:2.7.14

## General info

<b>Due date</b>: 11pm, Wednesday May 2nd

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character level language models, ranging from unigram approaches to higher over n-gram models. Your objective is to create an automatic player which makes the fewest mistakes.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks.  

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


## The Hangman Game

The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game, and a method allowing interactive play. 

In [1]:
# allowing better python 2 & python 3 compatibility 
# from __future__ import print_function 

def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of allowed mistakes
        verbose: be chatty vs silent
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print "Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word)
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print "You have", (max_mistakes-mistakes), "attempts remaining."
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print 'Guess is', guess
        if guess in guessed:
            if verbose:
                print 'Already guessed this before.'
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print 'Good guess:', ' '.join(mask)
            else:
                if verbose:
                    print 'Sorry, try again.'
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print 'Congratulations, you won.'
            return mistakes
        
    if verbose:
        print 'Out of guesses. The word was', secret_word
    return mistakes

def human(mask, guessed, **kwargs):
    """
    simple function for manual play
    """
    try:
        return raw_input().lower().strip() # python 3
    except NameError:
        return input().lower().strip() # python 2

You can play the game interactively using the following command:

In [5]:
hangman('whatever', human, 8, True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ length 8
You have 8 attempts remaining.
w
Guess is w
Good guess: w _ _ _ _ _ _ _
You have 8 attempts remaining.
a
Guess is a
Good guess: w _ a _ _ _ _ _
You have 8 attempts remaining.
e
Guess is e
Good guess: w _ a _ e _ e _
You have 8 attempts remaining.
r
Guess is r
Good guess: w _ a _ e _ e r
You have 8 attempts remaining.
h
Guess is h
Good guess: w h a _ e _ e r
You have 8 attempts remaining.
v
Guess is v
Good guess: w h a _ e v e r
You have 8 attempts remaining.
t
Guess is t
Good guess: w h a t e v e r
Congratulations, you won.


0

<b>Instructions</b>: We will be using the words occurring in the *Brown* corpus for *training* an artificial intelligence guessing algorithm, and for *evaluating* the quality of the method. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

Your first task is to compute the unique word types occurring in the *Brown* corpus, using `nltk.corpus.Brown`, selecting only words that are entirely comprised of alphabetic characters, and lowercasing the words. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the training set. Your code should print the sizes of the training and test sets.

Feel free to test your own Hangman performance using `hangman(numpy.random.choice(test_set), human, 8, True)`. It is surprisingly difficult (and addictive)!

(0.5 mark)

In [4]:
from nltk.corpus import brown
import numpy as np
typeList = list(set([word.lower() for para in brown.paras() for sent in para for word in sent if word.isalpha()]))
np.random.shuffle(typeList)
typeTrainList = typeList[:len(typeList)-1000]
typeTestList = typeList[len(typeList)-1000:]
print len(typeTrainList)
print len(typeTestList)



39234
1000


<b>Instructions</b>: To set a baseline, your first *AI* attempt will be a trivial random method. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `'a'...'z'` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses). You might want to use `numpy.random.choice` for this purpose.

To measure the performance of this (and later) techiques, implement a method that measures the average number of mistakes made by this technique over all the words in the `test_set`. You will want to turn off the printouts for this, using the `verbose=False` option, and increase the cap on the game length to `max_mistakes=26`. Print the average number of mistakes for the random AI, which will become a baseline for the following steps.

(1 mark)

In [3]:
import random
alphaSet = set("abcdefghijklmnopqrstuvwxyz")
def baselineGusser(mask, guessed, **kwargs):
    temps = kwargs["alphaSet"] - guessed
    return list(temps)[random.randint(0,len(temps)-1)]
total = 0
for secretWord in typeTestList:
    total+= hangman(secretWord, baselineGusser, 26, False,alphaSet=alphaSet)
print total/float(len(typeTestList))
    

16.841


**Instructions:** As your first real AI, you should train a *unigram* model over the training set.  This requires you to find the frequencies of characters over all training words. Using this model, you should write a guess function that returns the character with the highest probability, after aggregating (summing) the probability of each blank character in the secret word. Print the average number of mistakes the unigram method makes over the test set. Remember to exclude already guessed characters, and use the same evaluation method as above, so the results are comparable. (Hint: it should be much lower than for random).

(1 mark)

In [6]:
import collections
import random
import numpy as np
charProDic = collections.defaultdict(int)
for trainWord in typeTrainList:
    for c in trainWord:
        charProDic[c]+=1

alphaSet = set("abcdefghijklmnopqrstuvwxyz")
def unigramGusser(mask, guessed, **kwargs):
    temps = kwargs["alphaSet"] - guessed
    charProDic= kwargs["charProDic"]
    return max([(charProDic[c],c) for c in temps])[1]
total = 0
for secretWord in typeTestList:
    total+= hangman(secretWord, unigramGusser, 26, False,alphaSet=alphaSet,charProDic=charProDic)
print total/float(len(typeTestList))


    

10.253


**Instructions:** The length of the secret word is an important clue that we might exploit. Different length words tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. Your job now is to incorporate this idea by conditioning the unigram model on the length of the secret word, i.e., having *different* unigram models for each length of word. You will need to be a little careful at test time, to be robust to the (unlikely) situation that you encounter a word length that you didn't see in training. Create another AI guessing function using this new model, and print its test performance.   

(0.5 marks)

In [7]:
import collections
import random
import numpy as np
charLenProDic = {}
# charProDic = collections.defaultdict(int)
for trainWord in typeTrainList:
    charLenProDic.setdefault(len(trainWord),collections.defaultdict(int))
    for c in trainWord:
        charLenProDic[len(trainWord)][c]+=1

alphaSet = set("abcdefghijklmnopqrstuvwxyz")
def unigramLengthGusser(mask, guessed, **kwargs):
    temps = kwargs["alphaSet"] - guessed
    charLenProDic= kwargs["charLenProDic"]
    length = kwargs["length"]
    return max([(charLenProDic[length][c],c) for c in temps])[1]
total = 0
for secretWord in typeTestList:
    total+= hangman(secretWord, unigramLengthGusser, 26, False,alphaSet=alphaSet,charLenProDic=charLenProDic,length=len(secretWord))
print total/float(len(typeTestList))



10.193


**Instructions:** Now for the main challenge, using a *ngram* language model over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

Your job is to develop a *ngram* language model over characters, train this over the training words (being careful to handle the start of each word properly, e.g., by padding with sentinel symbols.) You should use linear interpolation to smooth between the higher order and lower order models, and you will have to decide how to weight each component. 

Your guessing AI algorithm should apply your language model to each blank position in the secret word by using as much of the left context as is known. E.g., in `_ e c _ e _ _` we know the full left context for the first blank (context=start of word), we have a context of two characters for the second blank (context=ec), one character for the second last blank (context=e), and no known context for the last one. If we were using a *n=3* order model, we would be able to apply it to the first and second blanks, but would only be able to use the bigram or unigram distributions for the subsequent blanks. As with the unigram model, you should sum over the probability distributions for each blank to find the expected count for each character type, then select the  character with the highest expected count.

Implement the ngram method for *n=3,4* and *5* and evaluate each of these three models over the test set. Do you see any improvement over the unigram methods above?

(2 marks)

In [5]:
import collections
import random
import numpy as np
        
    
uniGramCountDic = collections.defaultdict(int)
total = 0
for trainWord in typeTrainList:
    total +=(len(trainWord)+2)
    tempword = "*"+trainWord+"*"
    for i in range(len(tempword)-1):
        uniGramCountDic[tempword[i]]+=1
uniGramProDic = collections.defaultdict(int)
uniGramCondProDic = collections.defaultdict(int)
for c in uniGramCountDic:
    uniGramProDic[c] = uniGramCountDic[c]/float(total)
    uniGramCondProDic[c] = uniGramProDic[c]
    


twoGramCountDic = collections.defaultdict(int)
total = 0
for trainWord in typeTrainList:
    total += (len(tempword)+1)
    tempword = "*"+trainWord+"*"
    for i in range(len(tempword)-1):
        twoGramCountDic[tempword[i:i+2]]+=1
twoGramProDic = collections.defaultdict(int)
twoGramCondProDic = collections.defaultdict(int)
for c in twoGramCountDic:
    twoGramProDic[c] = twoGramCountDic[c]/float(total)
    twoGramCondProDic[c] = twoGramProDic[c]/uniGramProDic[c[0]]
# print twoGramProDic
    
total = 0
threeGramCountDic = collections.defaultdict(int)
for trainWord in typeTrainList:
    total += len(trainWord)
    tempword = "**"+trainWord+"**"
    for i in range(len(tempword)-2):
        threeGramCountDic[tempword[i:i+3]]+=1
threeGramProDic = collections.defaultdict(int)
threeGramCondProDic = collections.defaultdict(int)
for c in threeGramCountDic:
    if c[0:2] == "**":
        continue
    threeGramProDic[c] = threeGramCountDic[c]/float(total)
    threeGramCondProDic[c] = threeGramProDic[c]/twoGramProDic[c[0:2]]
        
total = 0
fourGramCountDic = collections.defaultdict(int)
for trainWord in typeTrainList:
    total += (len(trainWord)-1)
    tempword = "***"+trainWord+"***"
    for i in range(len(tempword)-3):
        fourGramCountDic[tempword[i:i+4]]+=1
fourGramProDic = collections.defaultdict(int)
fourGramCondProDic = collections.defaultdict(int)
for c in fourGramCountDic:
    if c[0:2] == "**":
        continue
    fourGramProDic[c] = fourGramCountDic[c]/float(total)
    fourGramCondProDic[c] = fourGramProDic[c]/threeGramProDic[c[0:3]]
        
total = 0
fiveGramCountDic = collections.defaultdict(int)
for trainWord in typeTrainList:
    total += (len(trainWord)-2)
    tempword = "****"+trainWord+"****"
    for i in range(len(tempword)-4):
        fiveGramCountDic[tempword[i:i+5]]+=1
fiveGramProDic = collections.defaultdict(int)
fiveGramCondProDic = collections.defaultdict(int)
for c in fiveGramCountDic:
    if c[0:2] == "**":
        continue
    fiveGramProDic[c] = fiveGramCountDic[c]/float(total)
    fiveGramCondProDic[c] = fiveGramProDic[c]/fourGramProDic[c[0:4]]
proDic = {}


proDic["uniGramCondProDic"] = uniGramCondProDic
proDic["twoGramCondProDic"] = twoGramCondProDic
proDic["threeGramCondProDic"] = threeGramCondProDic
proDic["fourGramCondProDic"] = fourGramCondProDic
proDic["fiveGramCondProDic"] = fiveGramCondProDic

nGramCondProDicList = []
nGramCondProDicList.append(uniGramCondProDic)
nGramCondProDicList.append(twoGramCondProDic)
nGramCondProDicList.append(threeGramCondProDic)
nGramCondProDicList.append(fourGramCondProDic)
nGramCondProDicList.append(fiveGramCondProDic)

alphaSet = set("abcdefghijklmnopqrstuvwxyz")


def ngramGusserMix2(mask, guessed, **kwargs):
    temps = kwargs["alphaSet"] - guessed
    n = kwargs["n"]
    nGramCondProDicList = kwargs["nGramCondProDicList"]
    tempmask = "*"+"".join(mask)+"*"
    m = collections.defaultdict(int)
    precount = 0
    for i in range(len(tempmask)-1):
        if tempmask[i] == "_":
            for c in temps:
                m[c]+=nGramCondProDicList[precount][tempmask[i-precount:i]+c]
            precount = 0
        else:
            precount+=1
            precount = min(precount,n-1)
    return max([(m[c],c) for c in temps])[1]

#add smoothing
def ngramGusserMix3(mask, guessed, **kwargs):
    temps = kwargs["alphaSet"] - guessed
    n = kwargs["n"]
    mylambda =  kwargs["mylambda"]
    nGramCondProDicList = kwargs["nGramCondProDicList"]
    tempmask = "*"+"".join(mask)+"*"
    m = collections.defaultdict(int)
    precount = 0
    for i in range(len(tempmask)-1):
        if tempmask[i] == "_":
            for c in temps:
                temp_lambda = 1
                for k in range(precount,-1,-1):
                    temp = nGramCondProDicList[k][tempmask[i-k:i]+c]
                    if temp > 0:
                        m[c]+= temp*mylambda*temp_lambda
                        break
                    temp_lambda = temp_lambda*(1-mylambda)
            precount = 0
        else:
            precount+=1
            precount = min(precount,n-1)
    return max([(m[c],c) for c in temps])[1]


# typeTestList3 = typeTestList[:50]

for n in range(3,6):
    total = 0
    for i,secretWord in enumerate(typeTestList):
        temp = hangman(secretWord, ngramGusserMix3, 26, False,alphaSet=alphaSet,nGramCondProDicList=nGramCondProDicList,n=n,mylambda = 0.5)
        total+=temp
    print str(n)+" gram :" +str(total/float(len(typeTestList)))










3 gram :8.346
4 gram :8.008
5 gram :7.803


## Bonus: Improving the AI

**Instructions:** To get the extra credit, you should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word and the parts that are known, fancier smoothing methods, using backwards ngram models, or a fancier inference algorithm like forward-back using a HMM. Ensure you report the test performance of your method.

You will be marked based on the ambition of your approach and on its accuracy. If you have tried some truly spectacular method but it didn't really work, then please include your implementation and an explanation, which will still attract marks for ambition.

(1 bonus mark) 

In [11]:
import collections
import random
import numpy as np
        
twoGramCountDic = collections.defaultdict(int)
for trainWord in typeTrainList:
    tempword = "*"+trainWord+"*"
    for i in range(len(tempword)-1):
        twoGramCountDic[tempword[i:i+2]]+=1

threeGramCountDic = collections.defaultdict(int)
for trainWord in typeTrainList:
    tempword = "**"+trainWord+"**"
    for i in range(len(tempword)-2):
        threeGramCountDic[tempword[i:i+3]]+=1

        
fourGramCountDic = collections.defaultdict(int)
for trainWord in typeTrainList:
    tempword = "***"+trainWord+"***"
    for i in range(len(tempword)-3):
        fourGramCountDic[tempword[i:i+4]]+=1
        
fiveGramCountDic = collections.defaultdict(int)
for trainWord in typeTrainList:
    tempword = "****"+trainWord+"****"
    for i in range(len(tempword)-4):
        fiveGramCountDic[tempword[i:i+5]]+=1

# print len(twoGramCountDic)
# print len(threeGramCountDic)
# print len(fourGramCountDic)
# print len(fiveGramCountDic)


twoGramCountTrie = collections.defaultdict(dict)
for trainWord in typeTrainList:
    tempword = "*"+trainWord+"*"
    for i in range(len(tempword)-1):
        temptrie = twoGramCountTrie[tempword[i]]
        temptrie[tempword[i+1]] = temptrie.get(tempword[i+1],0) +1

threeGramCountTrie = collections.defaultdict(dict)
for trainWord in typeTrainList:
    tempword = "**"+trainWord+"**"
    for i in range(len(tempword)-2):
        temptrie = threeGramCountTrie[tempword[i]]
        temptrie[tempword[i+1]] = temptrie.get(tempword[i+1],{})
        temptrie = temptrie[tempword[i+1]]
        temptrie[tempword[i+2]] = temptrie.get(tempword[i+2],0)+1
        

fourGramCountTrie = collections.defaultdict(dict)
for trainWord in typeTrainList:
    tempword = "***"+trainWord+"***"
    for i in range(len(tempword)-3):
        temptrie = fourGramCountTrie[tempword[i]]
        temptrie[tempword[i+1]] = temptrie.get(tempword[i+1],{})
        temptrie = temptrie[tempword[i+1]]
        temptrie[tempword[i+2]] = temptrie.get(tempword[i+2],{})
        temptrie = temptrie[tempword[i+2]]
        temptrie[tempword[i+3]] = temptrie.get(tempword[i+3],0)+1
             
        
fiveGramCountTrie = collections.defaultdict(dict)
for trainWord in typeTrainList:
    tempword = "****"+trainWord+"****"
    for i in range(len(tempword)-4):
        temptrie = fiveGramCountTrie[tempword[i]]
        temptrie[tempword[i+1]] = temptrie.get(tempword[i+1],{})
        temptrie = temptrie[tempword[i+1]]
        temptrie[tempword[i+2]] = temptrie.get(tempword[i+2],{})
        temptrie = temptrie[tempword[i+2]]
        temptrie[tempword[i+3]] = temptrie.get(tempword[i+3],{})
        temptrie = temptrie[tempword[i+3]] 
        temptrie[tempword[i+4]] = temptrie.get(tempword[i+4],0)+1
        
        
        
alphaSet = set("abcdefghijklmnopqrstuvwxyz")
def compareTwo(word1,word2):
    for i in range(len(word1)):
        if word2[i] != word1[i] and word2[i]!="_":
            return False;
    return True;
            
        
def twogramGusser(mask, guessed, **kwargs):
    temps = kwargs["alphaSet"] - guessed
    twoGramCountDic= kwargs["twoGramCountDic"]
    tempmask = "*"+"".join(mask)+"*"
    m = collections.defaultdict(int)
    for i in range(1,len(tempmask)):
        temp2gram = tempmask[i-1:i+1]
#         print temp2gram
        for target in twoGramCountDic.keys():
            if compareTwo(target,temp2gram):
                m[target[0]]+= twoGramCountDic[target]
                m[target[1]]+= twoGramCountDic[target]
#     print m
    return max([(m[c],c) for c in temps])[1]



def twogramGusser2(mask, guessed, **kwargs):
    temps = kwargs["alphaSet"] - guessed
    twoGramCountTrie= kwargs["twoGramCountTrie"]
    tempmask = "*"+"".join(mask)+"*"
    m = collections.defaultdict(int)
    for i in range(len(tempmask)-1):
        if tempmask[i] == "_":
            for key1 in twoGramCountTrie.keys():
                temptrie = twoGramCountTrie[key1]
                if tempmask[i+1] == "_":
                    for key2 in temptrie.keys():
                        m[key1]+=temptrie[key2]
                        m[key2]+=temptrie[key2]
                else:
                    key2 = tempmask[i+1]
                    m[key1]+= temptrie.get(key2,0)
                    m[key2]+= temptrie.get(key2,0)
        else:
            key1 = tempmask[i]
            temptrie = twoGramCountTrie.get(key1,{})
            if tempmask[i+1] == "_":
                for key2 in temptrie.keys():
                    m[key1]+=temptrie[key2]
                    m[key2]+=temptrie[key2]
            else:
                key2 = tempmask[i+1]
                m[key1]+= temptrie.get(key2,0)
                m[key2] += temptrie.get(key2,0)
    return max([(m[c],c) for c in temps])[1]

def handle(tempmask,index,trie,m,keyList,remainLevel):
    if remainLevel == 1:
        if tempmask[index] == "_":
            for keyt in trie.keys():
                for key in keyList:
                    m[key]+=trie[keyt]
                m[keyt] += trie[keyt]
        else:
            keyt = tempmask[index]
            for key in keyList:
                m[key]+=trie.get(keyt,0)
            m[keyt]+=trie.get(keyt,0)
    else:
        if tempmask[index] == "_":
            for key2 in trie.keys():
                temptrie = trie[key2]
                handle(tempmask,index+1,temptrie,m,keyList+[key2],remainLevel-1)
        else:
            key2 = tempmask[index]
            temptrie = trie.get(key2,{})
            handle(tempmask,index+1,temptrie,m,keyList+[key2],remainLevel-1)
            
def twogramGusser3(mask, guessed, **kwargs):
    temps = kwargs["alphaSet"] - guessed
    twoGramCountTrie= kwargs["twoGramCountTrie"]
    tempmask = "*"+"".join(mask)+"*"
    m = collections.defaultdict(int)
    for i in range(len(tempmask)-1):
        handle(tempmask,i,twoGramCountTrie,m,[],2)
    return max([(m[c],c) for c in temps])[1]

def threegramGusser3(mask, guessed, **kwargs):
    temps = kwargs["alphaSet"] - guessed
    threeGramCountTrie= kwargs["threeGramCountTrie"]
    tempmask = "**"+"".join(mask)+"**"
    m = collections.defaultdict(int)
    for i in range(len(tempmask)-2):
        handle(tempmask,i,threeGramCountTrie,m,[],3)
    return max([(m[c],c) for c in temps])[1]

def fourgramGusser3(mask, guessed, **kwargs):
    temps = kwargs["alphaSet"] - guessed
    fourGramCountTrie= kwargs["fourGramCountTrie"]
    tempmask = "***"+"".join(mask)+"***"
    m = collections.defaultdict(int)
    for i in range(len(tempmask)-3):
        handle(tempmask,i,fourGramCountTrie,m,[],4)
    return max([(m[c],c) for c in temps])[1]
def fivegramGusser3(mask, guessed, **kwargs):
    temps = kwargs["alphaSet"] - guessed
    fiveGramCountTrie= kwargs["fiveGramCountTrie"]
    tempmask = "****"+"".join(mask)+"****"
    m = collections.defaultdict(int)
    for i in range(len(tempmask)-4):
        handle(tempmask,i,fiveGramCountTrie,m,[],5)
    return max([(m[c],c) for c in temps])[1]
np.random.shuffle(typeTestList)
typeTestList2 = typeTestList[0:10]

print "because my method is slow, So I choose just 50 testcase to check"
print ""
print "origin version ------------------"
for n in range(3,6):
    total = 0
    for i,secretWord in enumerate(typeTestList2):
        temp = hangman(secretWord, ngramGusserMix3, 26, False,alphaSet=alphaSet,nGramCondProDicList=nGramCondProDicList,n=n,mylambda = 0.5)
        total+=temp
    print str(n)+" gram :" +str(total/float(len(typeTestList2)))
print ""
print "we know the origin version _b_ _ we can just guess the second blank from b, but actually, the first and the third blank should also related to the character \"b\""
print "then my improvement try to use all the information between not only the left char but also the right char."
print "And my improvement also consider the character's effect on the unadjacented blank"
print ""
print ""
print "improve version -------------------"
total = 0
for i,secretWord in enumerate(typeTestList2):
    temp = hangman(secretWord, twogramGusser3, 26, False,alphaSet=alphaSet,twoGramCountTrie=twoGramCountTrie)
    print i,temp,secretWord
    total+=temp
print "2gram : " +str(total/float(len(typeTestList2)))
print  "------------------------------------"
print ""

total = 0
for i,secretWord in enumerate(typeTestList2):
    temp = hangman(secretWord, threegramGusser3, 26, False,alphaSet=alphaSet,threeGramCountTrie=threeGramCountTrie)
    print i,temp,secretWord
    total+=temp
print "3gram : " +str(total/float(len(typeTestList2)))
print  "------------------------------------"
print ""

total = 0
for i,secretWord in enumerate(typeTestList2):
    temp = hangman(secretWord, fourgramGusser3, 26, False,alphaSet=alphaSet,fourGramCountTrie=fourGramCountTrie)
    print i,temp,secretWord
    total+=temp
print "4gram : " +str(total/float(len(typeTestList2)))
print  "------------------------------------"
print ""

total = 0
for i,secretWord in enumerate(typeTestList2):
    temp = hangman(secretWord, fivegramGusser3, 26, False,alphaSet=alphaSet,fiveGramCountTrie=fiveGramCountTrie)
    print i,temp,secretWord
    total+=temp
print "5gram : " +str(total/float(len(typeTestList2)))
print  "------------------------------------"






because my method is slow, So I choose just 50 testcase to check

origin version ------------------
3 gram :6.8
4 gram :6.4
5 gram :6.6

we know the origin version _b_ _ we can just guess the second blank from b, but actually, the first and the third blank should also related to the character "b"
then my improvement try to use all the information between not only the left char but also the right char.
And my improvement also consider the character's effect on the unadjacented blank


improve version -------------------
0 6 spate
1 8 schoolroom
2 9 archeological
3 9 intensifiers
4 14 magwitch
5 2 anson
6 4 chattels
7 4 rarity
8 13 overlooked
9 4 reproducible
2gram : 7.3
------------------------------------

0 6 spate
1 8 schoolroom
2 4 archeological
3 3 intensifiers
4 12 magwitch
5 1 anson
6 4 chattels
7 4 rarity
8 7 overlooked
9 4 reproducible
3gram : 5.3
------------------------------------

0 4 spate
1 8 schoolroom
2 4 archeological
3 1 intensifiers
4 8 magwitch
5 1 anson
6 4 chattel

In [ ]:
# def threegramGusser(mask, guessed, **kwargs):
#     temps = kwargs["alphaSet"] - guessed
#     threeGramCountDic= kwargs["threeGramCountDic"]
#     tempmask = "**"+"".join(mask)+"**"
#     m = collections.defaultdict(int)
#     for i in range(len(tempmask)-2):
#         temp3gram = tempmask[i:i+3]
#         for target in threeGramCountDic.keys():
#             if compareTwo(target,temp3gram):
#                 m[target[0]]+= threeGramCountDic[target]
#                 m[target[1]]+= threeGramCountDic[target]
#                 m[target[2]]+= threeGramCountDic[target]
#     return max([(m[c],c) for c in temps])[1]

# def fourgramGusser(mask, guessed, **kwargs):
#     temps = kwargs["alphaSet"] - guessed
#     fourGramCountDic= kwargs["fourGramCountDic"]
#     tempmask = "***"+"".join(mask)+"***"
#     m = collections.defaultdict(int)
#     for i in range(len(tempmask)-3):
#         temp4gram = tempmask[i:i+4]
#         for target in fourGramCountDic.keys():
#             if compareTwo(target,temp4gram):
#                 m[target[0]]+= fourGramCountDic[target]
#                 m[target[1]]+= fourGramCountDic[target]
#                 m[target[2]]+= fourGramCountDic[target]
#                 m[target[3]]+= fourGramCountDic[target]
#     return max([(m[c],c) for c in temps])[1]

# def fivegramGusser(mask, guessed, **kwargs):
#     temps = kwargs["alphaSet"] - guessed
#     fiveGramCountDic= kwargs["fiveGramCountDic"]
#     tempmask = "***"+"".join(mask)+"***"
#     m = collections.defaultdict(int)
#     for i in range(len(tempmask)-4):
#         temp5gram = tempmask[i:i+5]
#         for target in fiveGramCountDic.keys():
#             if compareTwo(target,temp5gram):
#                 m[target[0]]+= fiveGramCountDic[target]
#                 m[target[1]]+= fiveGramCountDic[target]
#                 m[target[2]]+= fiveGramCountDic[target]
#                 m[target[3]]+= fiveGramCountDic[target]
#                 m[target[4]]+= fiveGramCountDic[target]
#     return max([(m[c],c) for c in temps])[1]


# total = 0
# for i,secretWord in enumerate(typeTestList):
#     temp = hangman(secretWord, threegramGusser, 26, False,alphaSet=alphaSet,threeGramCountDic=threeGramCountDic)
# #     temp = hangman(secretWord, twogramGusser2, 26, False,alphaSet=alphaSet,twoGramCountTrie=twoGramCountTrie)
#     total+=temp
#     print i,temp,secretWord
#     if i % 10==9:    
# #         print i,temp
#         break
# # print total/float(len(typeTestList))
# print "3gram : "+str(total/10.0)


# # total = 0
# # for i,secretWord in enumerate(typeTestList):
# #     temp = hangman(secretWord, fourgramGusser, 26, False,alphaSet=alphaSet,fourGramCountDic=fourGramCountDic)
# # #     temp = hangman(secretWord, twogramGusser2, 26, False,alphaSet=alphaSet,twoGramCountTrie=twoGramCountTrie)
# #     total+=temp
# #     if i % 10==9:    
# #         print i,temp
# #         break
# # # print total/float(len(typeTestList))
# # print "4gram : "+str(total/10.0)


# # total = 0
# # for i,secretWord in enumerate(typeTestList):
# #     temp = hangman(secretWord, fivegramGusser, 26, False,alphaSet=alphaSet,fiveGramCountDic=fiveGramCountDic)
# # #     temp = hangman(secretWord, twogramGusser2, 26, False,alphaSet=alphaSet,twoGramCountTrie=twoGramCountTrie)
# #     total+=temp
# #     print "111111111111"
# #     if i % 10==9:    
# #         print i,temp
# #         break
# # # print total/float(len(typeTestList))
# # print "5gram : "+str(total/10.0)







# # print total/100.0





